<a href="https://colab.research.google.com/github/Anson2Leung/Anson2Leung.github.io/blob/main/assignment_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!git clone https://github.com/Anson2Leung/ece405-assignment1-basics.git

Cloning into 'ece405-assignment1-basics'...
remote: Enumerating objects: 465, done.
remote: Counting objects: 100% (465/465), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 465 (delta 241), reused 461 (delta 240), pack-reused 0 (from 0)
Receiving objects: 100% (465/465), 22.46 MiB | 29.34 MiB/s, done.
Resolving deltas: 100% (241/241), done.


In [2]:
# Install uv
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Add uv to the system path so you can use it directly
import os
os.environ['PATH'] = f"{os.path.expanduser('~/.cargo/bin')}:{os.environ['PATH']}"

downloading uv 0.9.28 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [2]:
%cd ece405-assignment1-basics

!uv pip install -r requirements.txt --system
!pip install "numpy<2.0" jaxtyping typeguard langsmith

/content/ece405-assignment1-basics
Using Python 3.12.12 environment at: /usr
Audited 3 packages in 110ms


# Test run

In [3]:
!pytest tests/test_train_bpe.py

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0
rootdir: /content/ece405-assignment1-basics
configfile: pytest.ini
plugins: jaxtyping-0.3.7, anyio-4.12.1, typeguard-4.4.4, langsmith-0.6.6
collected 3 items                                                              

tests/test_train_bpe.py::test_train_bpe_speed PASSED                     [ 33%]
tests/test_train_bpe.py::test_train_bpe PASSED                           [ 66%]
tests/test_train_bpe.py::test_train_bpe_special_tokens FAILED            [100%]

=================================== FAILURES ===================================
________________________ test_train_bpe_special_tokens _________________________

snapshot = <tests.conftest.Snapshot object at 0x7dcb42290d10>

    def test_train_bpe_special_tokens(snapshot):
        """
        Ensure that the special tokens are added to the vocabulary and not
        merged with other tokens.


# Problem (train_bpe_tinystories): BPE Training on TinyStories (2 points)


In [4]:
%cd ..
%mkdir -p data
%cd data

!wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt
!wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-valid.txt

!wget https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_train.txt.gz
!gunzip owt_train.txt.gz
!wget https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_valid.txt.gz
!gunzip owt_valid.txt.gz

%cd ..
%cd ece405-assignment1-basics

/content
/content/data
--2026-02-03 04:19:49--  https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt
Resolving huggingface.co (huggingface.co)... 3.167.112.25, 3.167.112.38, 3.167.112.45, ...
Connecting to huggingface.co (huggingface.co)|3.167.112.25|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/645e8da96320b0efe40ade7a/02e40cc51c59a4bc6c51bd7bc9acda4316e208745be060558eaf500cd14e9f96?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27TinyStoriesV2-GPT4-train.txt%3B+filename%3D%22TinyStoriesV2-GPT4-train.txt%22%3B&response-content-type=text%2Fplain&Expires=1770095990&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzcwMDk1OTkwfX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjQ1ZThkYTk2MzIwYjBlZmU0MGFkZTdhLzAyZTQwY2M1MWM1OWE0YmM2YzUxYmQ3YmM5YWNkYTQzMTZlMjA4NzQ1YmUwNjA1NThlYWY1MDBjZDE0ZTlmOTZcXD9yZXNwb25zZ

In [8]:
import sys
import os
import json

# 1. Ensure the script is in the path
%cd /content/ece405-assignment1-basics
sys.path.append('/content/ece405-assignment1-basics/assignment_files')
from bpe_tokenizer import train_bpe, save_vocab, save_merges

# 2. Configuration
INPUT_PATH = "/content/data/TinyStoriesV2-GPT4-train.txt"
VOCAB_SIZE = 10000
SPECIAL_TOKENS = ["<|endoftext|>"]

# 3. Execution
# We call the function directly. Your code already handles the timing and memory printing.
vocab, merges = train_bpe(INPUT_PATH, VOCAB_SIZE, SPECIAL_TOKENS)

# 4. Serialization
save_vocab(vocab, "vocab.json")
save_merges(merges, "merges.txt")

/content/ece405-assignment1-basics
Dataset:  /content/data/TinyStoriesV2-GPT4-train.txt
Number of processes requested:  2
Special tokens as bytes:  [b'<|endoftext|>']
Total number of tasks to complete:  1


KeyboardInterrupt: 

In [ ]:
# Analysis
def get_decoded_string(token_bytes):
    return token_bytes.decode('utf-8', errors='ignore')

# Find the longest token in the vocabulary
longest_token_id = max(vocab, key=lambda k: len(get_decoded_string(vocab[k])))
longest_token_bytes = vocab[longest_token_id]
longest_text = get_decoded_string(longest_token_bytes)

print(f"--- Analysis Results ---")
print(f"Longest Token ID: {longest_token_id}")
print(f"Longest Token Text: '{longest_text}'")
print(f"Character Length: {len(longest_text)}")